# Activity 01 - Instructor Turn - Ins Joins - 👩‍🏫🧑‍🏫 






## Setup

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

In [ ]:
# Create Database Connection
engine = create_engine("sqlite:///./resources/mammal_masses.sqlite", echo=False)

In [ ]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(autoload_with=engine)
Base.classes.keys()

In [ ]:
# Map Europe class
EA = Base.classes.ea
# Map North American class
NA = Base.classes.na
# create a session
session = Session(engine)


Filtering Review
Filters are the "WHERE" clause for your select statement.


In [ ]:
# Filter North American mammals whose genus is "Antilocapra"
# Query, loop over and print out animals.
mammals = session.query(NA).filter(NA.genus == 'Antilocapra').all()
for mammal in mammals:
    print(f"Family: {mammal.family}, Genus: {mammal.genus}")


Joins
A SQL join combines columns from one or more tables in a relational database.

It creates a set that can be saved as a table or used as it is.

A JOIN is a means for combining columns from one (self-table) or more tables by using values common to each.


In [ ]:
# Get the table names using `inspect()`.
inspector = inspect(engine)
inspector.get_table_names()



In [ ]:
# Get a list of column names and types
columns = inspector.get_columns('ea')
for c in columns:
    print(c['name'], c["type"])



In [ ]:
# Join all the order names for the EA and NA classes. 
# This returns a warning so the query should be filtered on a common order name.
session.query(EA.order, NA.order).limit(200).all()



In [ ]:
# Filter the the join query. 
same_order = session.query(EA, NA).filter(EA.order == NA.order).limit(10).all()

for record in same_order:
    (ea, na) = record
    print(ea.order, ea.family, ea.genus, ea.species)
    print()
    print(na.order, na.family, na.genus, na.species)


In [ ]:
# Return all animals from EA and NA belonging to the same order.
# This JOINs the data in the two tables together into a single dataset (here in the form of a tuple).
# Note: We are going to limit the results to 10 for printing
sel = [EA.family, EA.genus, EA.species, NA.family, NA.genus, NA.species]
same_order = session.query(*sel).filter(EA.order == NA.order).limit(10).all()


In [ ]:
for record in same_order:
    (ea_fam, ea_gen, ea_spec, na_fam, na_gen, na_spec) = record
    print(
        f"The European animal '{ea_fam} {ea_gen} {ea_spec}'"
        f"belongs to the same order as the North American animal '{na_fam} {na_gen} {na_spec}'.")


# Activity 02  - Instructor Turn - Dates - 👩‍🏫🧑‍🏫 

In [ ]:
# Setup
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, text

In [ ]:
engine = create_engine("sqlite:///./resources/dow.sqlite", echo=False)

Explore Database

In [ ]:

inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
# Get a list of column names and types
columns = inspector.get_columns('dow')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM dow LIMIT 5;")).fetchall()
    print(result)

In [ ]:
engine.execute('SELECT * FROM dow LIMIT 5').fetchall()

In [ ]:
# Reflect and query dates
# Reflect Database into ORM class
Base = automap_base()
Base.prepare(autoload_with=engine)
Dow = Base.classes.dow
session = Session(engine)

In [ ]:
# Total dates
session.query(func.count(Dow.date)).all()

In [ ]:
# Earliest Date
session.query(Dow.date).order_by(Dow.date).first()

In [ ]:

# Latest Date
session.query(Dow.date).order_by(Dow.date.desc()).first()

In [ ]:
# Find all of the dates great than `2011-03-01`
session.query(Dow.date).\
    filter(Dow.date > '2011-03-01').\
    order_by(Dow.date).all()

**Important Note! Sqlite does not support a date column type, but SQLAlchemy will allow you to work with dates in the iso format. sqlite dates
Quick Review of DateTime**










In [ ]:
import datetime as dt
# Print a date object and a datetime object 
print(dt.date.today())
print(dt.date(2017, 1 ,31))

print(dt.datetime.utcnow())
print(dt.datetime(2017, 1, 31))

In [ ]:
# Calculate a time difference 1 week ago from today
week_ago = dt.date.today() - dt.timedelta(days=7)
week_ago


In [ ]:
# Query for the Dow closing price for `CSCO` 1 week before `2011-04-08` using the datetime library
query_date = dt.date(2011, 4, 8) - dt.timedelta(days=7)


In [ ]:
print("Query Date: ", query_date)
session.query(Dow.date, Dow.close_price).\
    filter(Dow.stock == 'CSCO').\
    filter(Dow.date == query_date).all()

In [ ]:
# Parse out just the day from the datetime object
dt.date.today().strftime("%d")

In [ ]:
# Query for all dates matching the 
# following date string in the format `%d`
date_str = "14"
session.query(Dow.date).\
    filter(func.strftime("%d", Dow.date) == date_str).all()


# Activity 03 - Student Turn - Dow Dates - 👩‍🎓👨‍🎓

# Dates

In this activity, you will practice working with dates, both in SQLAlchemy and with the `datetime` library.

## Instructions

* Use the provided `dow.sqlite` dataset to analyze the average stock prices (average open, average high, average low, and average close) for all stocks in the month of May.

* Plot the results as a Pandas or Matplotlib bar chart.

## Bonus

* Calculate the high-low peak-to-peak (PTP) values for `IBM` stock after `2011-05-31`.

    * The high-low PTP is calculated by subtracting `low_price` from `high_price`.

* Use a `DateTime.date` object in the query filter.

* Use list comprehension to create a list of dictionaries from the query results.

* Create a DataFrame from the list of dictionaries.

* Use the `boxplot()` method on the DataFrame to plot PTP distribution statistics.

---

## References

Brown, Michael. (2013). Weekly Dow Jones Index Data. 10.13140/2.1.2729.4409.

---

In [ ]:
# Import dependencies.
import matplotlib
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
engine = create_engine("sqlite:///./resources/dow.sqlite", echo=False)
engine.execute('SELECT * FROM dow LIMIT 5').fetchall()



In [ ]:
inspector = inspect(engine)
columns = inspector.get_columns('dow')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
# Reflect Database into ORM class


## Analysis
## Analyze the Average prices (open, high, low, close) for all stocks in the Month of May



In [ ]:
# Query for the stock and average prices (open, high, low, close) 
# for all stock in the month of May
# Sort the result by stock name


In [ ]:
# Plot the Results in a Matplotlib bar chart


# **Bonus**
Calculate the high-low peak-to-peak (PTP) values for IBM stock after 2011-05-31.

Note: high-low PTP is calculated using high_price - low_price
Use a DateTime.date object in the query filter
Use list comprehension to create a list of dictionaries from the query results
Create a dataframe from the list of dictionaries
Use the boxplot() method on the dataframe to plot PTP distribution statistics



In [ ]:
# Design a query to calculate the PTP for stock `IBM` after May, 2011


In [ ]:
# List comprehension solution



In [ ]:
# Load the list comprehension rows into a dataframe, set the index to the date, and plot the PTPs


<details>
    <summary><strong>✅ Solution 03 Click HERE</strong></summary>

```python


# Import dependencies.
import matplotlib
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
engine = create_engine("sqlite:///./resources/dow.sqlite", echo=False)
engine.execute('SELECT * FROM dow LIMIT 5').fetchall()

inspector = inspect(engine)
columns = inspector.get_columns('dow')
for c in columns:
    print(c['name'], c["type"])

# Reflect Database into ORM class
Base = automap_base()
Base.prepare(autoload_with=engine)
Dow = Base.classes.dow
session = Session(engine)

# Query for the stock and average prices (open, high, low, close) 
# for all stock in the month of May
# Sort the result by stock name
sel = [Dow.stock, 
       func.avg(Dow.open_price), 
       func.avg(Dow.high_price), 
       func.avg(Dow.low_price), 
       func.avg(Dow.close_price)]
may_averages = session.query(*sel).\
    filter(func.strftime("%m", Dow.date) == "05").\
    group_by(Dow.stock).\
    order_by(Dow.stock).all()
may_averages


# Plot the Results in a Matplotlib bar chart
df = pd.DataFrame(may_averages, columns=['stock', 'open_avg', 'high_avg', 'low_avg', 'close_avg'])
df.set_index('stock', inplace=True)
df.plot.bar()
plt.tight_layout()
plt.show()


# Design a query to calculate the PTP for stock `IBM` after May, 2011
import datetime as dt

date = dt.datetime(2011, 5, 31)

results = session.query(Dow.high_price - Dow.low_price, 
                        Dow.date).\
                  filter(Dow.date > date).filter(Dow.stock == 'IBM').all()

# List comprehension solution
ptp_rows = [{"Date": result[1], "PTP": result[0]} for result in results]
ptp_rows

# Load the list comprehension rows into a dataframe, set the index to the date, and plot the PTPs
pd.DataFrame(ptp_rows).set_index("Date").boxplot(patch_artist=True)
plt.title("IBM PTPs")
plt.show()

 
    
```


</details>

# Activity 04 -  Instructor Turn  - First Steps with Flask - 👩‍🏫🧑‍🏫 

# We will be doing flask for Activity 4 to 10

## Step 0 Setup
must run on either anaconda prompt for windows 
or terminal for Mac

install flask in your system
```
conda install -c anaconda flask
```

## Step 1 navigate to the file  
**cd** to the file location where the app.py is located.
Provided a folder called **flask_activities**

![Screenshot](./images/screenshot.png)

## Step 2. run the python file
```bash
python app.py
```

```bash
python3 app.py
```


```python
# 1. import Flask
from flask import Flask

# 2. Create an app, being sure to pass __name__
app = Flask(__name__)


# 3. Define what to do when a user hits the index route
@app.route("/")
def home():
    print("Server received request for 'Home' page...")
    return "Welcome to my 'Home' page!"


# 4. Define what to do when a user hits the /about route
@app.route("/about")
def about():
    print("Server received request for 'About' page...")
    return "Welcome to my 'About' page!"


if __name__ == "__main__":
    app.run(debug=True)
```

# Activity 05 - Student Turn - Hello Web - 👩‍🎓👨‍🎓


# Hello, Web

In this activity, you will practice setting up a server and defining basic routes with Flask.

## Instructions

* Create an `app.py`, and make the necessary imports.

* Use Flask to create an `app` instance.

* Use route decorators to define the following endpoints:

    * `/`, or your **index route**: This should return a simple string, such as `"Hello, world!"` or `"Welcome to my API!"`

    * `/about`, which should return a string containing your **name** and **current location**

    * `/contact`, which should return a string telling visitors where to email you

* Finally, add code at the end of the file that will allow you to run the server from the command line with `python app.py`.

## Hint

Refer to the [Flask documentation](http://flask.pocoo.org/docs/0.12/quickstart/#a-minimal-application) as you work through this activity.

---

<details>
    <summary><strong>✅ Solution 04 Click HERE</strong></summary>

```python

# 1. Import Flask
from flask import Flask


# 2. Create an app
app = Flask(__name__)


# 3. Define static routes
@app.route("/")
def index():
    return "Hello, world!"


@app.route("/about")
def about():
    name = "Peleke"
    location = "Tien Shan"

    return f"My name is {name}, and I live in {location}."


@app.route("/contact")
def contact():
    email = "peleke@example.com"

    return f"Questions? Comments? Complaints? Shoot an email to {email}."


# 4. Define main behavior
if __name__ == "__main__":
    app.run(debug=True)
```
</details>

# Activity 06 -  Instructor Turn  - Jsonify - 👩‍🏫🧑‍🏫 



```python


# Import the dependencies.
from flask import Flask, jsonify

# Create an app, being sure to pass __name__
app = Flask(__name__)

# Create a dictionary to hold a key, value pair.
hello_dict = {"Hello": "World!"}

# Define what to do when a user hits the index route.
@app.route("/")
def home():
    return "Hi"

# Define what to do when a user hits the /normal route
@app.route("/normal")
def normal():
    return hello_dict

# Define what to do when a user hits the /jsonified route
@app.route("/jsonified")
def jsonified():
    return jsonify(hello_dict)


if __name__ == "__main__":
    app.run(debug=True)
```


# Activity 07 - Student Turn - Justice League jsonify - 👩‍🎓👨‍🎓



# Justice League API

* In this activity, you will create an API route that returns the superhero name _and_ real name for every member of the Justice League.

## Instructions

* You may create a file called `app.py` for your Flask app, or you may use the provided starter code.

* If you did not choose to use the starter code, define a Python dictionary containing the superhero name and real name for each member of the Justice League (Superman, Batman, Wonder Woman, Green Lantern, Flash, Aquaman, and Cyborg).

    * You can gather that information from the Justice League Members [Wikipedia page](https://en.wikipedia.org/wiki/List_of_Justice_League_members).

    * Only gather the information for the 7 characters just listed.

* Create a **get** route called `/api/v1.0/justice-league`.

    * Inside of your **get** route, create a function called `justice_league` that will use `jsonify` to convert the dictionary of Justice League members to a JSON object and return that data as a request.

* Define a root route, `/`, that will return the usage statement for your API.

---


<details>
    <summary><strong>✅ Solution 07 Click HERE</strong></summary>

```python
from flask import Flask, jsonify

# Dictionary of Justice League
justice_league_members = [
    {"superhero": "Aquaman", "real_name": "Arthur Curry"},
    {"superhero": "Batman", "real_name": "Bruce Wayne"},
    {"superhero": "Cyborg", "real_name": "Victor Stone"},
    {"superhero": "Flash", "real_name": "Barry Allen"},
    {"superhero": "Green Lantern", "real_name": "Hal Jordan"},
    {"superhero": "Superman", "real_name": "Clark Kent/Kal-El"},
    {"superhero": "Wonder Woman", "real_name": "Princess Diana"}
]

#################################################
# Flask Setup
#################################################
app = Flask(__name__)


#################################################
# Flask Routes
#################################################

@app.route("/api/v1.0/justice-league")
def justice_league():
    """Return the justice league data as json"""

    return jsonify(justice_league_members)


@app.route("/")
def welcome():
    return (
        f"Welcome to the Justice League API!<br/>"
        f"Available Routes:<br/>"
        f"/api/v1.0/justice-league"
    )


if __name__ == "__main__":
    app.run(debug=True)

```

</details>

# Activity 08 -  Instructor Turn  - Variable Rule - 👩‍🏫🧑‍🏫 



```python
from flask import Flask, jsonify

justice_league_members = [
    {"superhero": "Aquaman", "real_name": "Arthur Curry"},
    {"superhero": "Batman", "real_name": "Bruce Wayne"},
    {"superhero": "Cyborg", "real_name": "Victor Stone"},
    {"superhero": "Flash", "real_name": "Barry Allen"},
    {"superhero": "Green Lantern", "real_name": "Hal Jordan"},
    {"superhero": "Superman", "real_name": "Clark Kent/Kal-El"},
    {"superhero": "Wonder Woman", "real_name": "Princess Diana"}
]

#################################################
# Flask Setup
#################################################
app = Flask(__name__)


#################################################
# Flask Routes
#################################################

@app.route("/api/v1.0/justice-league")
def justice_league():
    """Return the justice league data as json"""

    return jsonify(justice_league_members)


@app.route("/")
def welcome():
    return (
        f"Welcome to the Justice League API!<br/>"
        f"Available Routes:<br/>"
        f"/api/v1.0/justice-league<br/>"
        f"/api/v1.0/justice-league/Arthur%20Curry<br/>"
        f"/api/v1.0/justice-league/Bruce%20Wayne<br/>"
        f"/api/v1.0/justice-league/Victor%20Stone<br/>"
        f"/api/v1.0/justice-league/Barry%20Allen<br/>"
        f"/api/v1.0/justice-league/Hal%20Jordan<br/>"
        f"/api/v1.0/justice-league/Clark%20Kent/Kal-El<br/>"
        f"/api/v1.0/justice-league/Princess%20Diana"
    )


@app.route("/api/v1.0/justice-league/<real_name>")
def justice_league_character(real_name):
    """Fetch the Justice League character whose real_name matches
       the path variable supplied by the user, or a 404 if not."""

    canonicalized = real_name.replace(" ", "").lower()
    for character in justice_league_members:
        search_term = character["real_name"].replace(" ", "").lower()

        if search_term == canonicalized:
            return jsonify(character)

    return jsonify({"error": f"Character with real_name {real_name} not found."}), 404


if __name__ == "__main__":
    app.run(debug=True)

```


# Activity 09 - Student Turn - Variable Rule - 👩‍🎓👨‍🎓



# Routes with Variable Rules

In this activity, you will add an API route that returns a JSON of a Python dictionary containing an individual superhero’s information.

## Instructions

Using the previous activity as a starting point, add code to get a specific hero's information based on their superhero name.

---

<details>
    <summary><strong>✅ Solution 09 Click HERE</strong></summary>

```python
from flask import Flask, jsonify

justice_league_members = [
    {"superhero": "Aquaman", "real_name": "Arthur Curry"},
    {"superhero": "Batman", "real_name": "Bruce Wayne"},
    {"superhero": "Cyborg", "real_name": "Victor Stone"},
    {"superhero": "Flash", "real_name": "Barry Allen"},
    {"superhero": "Green Lantern", "real_name": "Hal Jordan"},
    {"superhero": "Superman", "real_name": "Clark Kent/Kal-El"},
    {"superhero": "Wonder Woman", "real_name": "Princess Diana"}
]

#################################################
# Flask Setup
#################################################
app = Flask(__name__)


#################################################
# Flask Routes
#################################################

@app.route("/api/v1.0/justice-league")
def justice_league():
    """Return the justice league data as json"""

    return jsonify(justice_league_members)


@app.route("/")
def welcome():
    return (
        f"Welcome to the Justice League API!<br/>"
        f"Available Routes:<br/>"
        f"/api/v1.0/justice-league<br/>"
        f"/api/v1.0/justice-league/superhero/batman<br/>"
        f"/api/v1.0/justice-league/real_name/bruce%20wayne"
    )


@app.route("/api/v1.0/justice-league/real_name/<real_name>")
def justice_league_by_real_name(real_name):
    """Fetch the Justice League character whose real_name matches
       the path variable supplied by the user, or a 404 if not."""

    canonicalized = real_name.replace(" ", "").lower()
    for character in justice_league_members:
        search_term = character["real_name"].replace(" ", "").lower()

        if search_term == canonicalized:
            return jsonify(character)

    return jsonify({"error": f"Character with real_name {real_name} not found."}), 404


@app.route("/api/v1.0/justice-league/superhero/<superhero>")
def justice_league_by_superhero__name(superhero):
    """Fetch the Justice League character whose superhero matches
       the path variable supplied by the user, or a 404 if not."""

    canonicalized = superhero.replace(" ", "").lower()
    for character in justice_league_members:
        search_term = character["superhero"].replace(" ", "").lower()

        if search_term == canonicalized:
            return jsonify(character)

    return jsonify({"error": "Character not found."}), 404


if __name__ == "__main__":
    app.run(debug=True)

```
</details>

# Activity 10 -  Instructor Turn  - Flask with ORM - 👩‍🏫🧑‍🏫 



```python
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify


#################################################
# Database Setup
#################################################
engine = create_engine("sqlite:///titanic.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(autoload_with=engine)

# Save reference to the table
Passenger = Base.classes.passenger

#################################################
# Flask Setup
#################################################
app = Flask(__name__)


#################################################
# Flask Routes
#################################################

@app.route("/")
def welcome():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/names<br/>"
        f"/api/v1.0/passengers"
    )


@app.route("/api/v1.0/names")
def names():
    # Create our session (link) from Python to the DB
    session = Session(engine)

    """Return a list of all passenger names"""
    # Query all passengers
    results = session.query(Passenger.name).all()

    session.close()

    # Convert list of tuples into normal list
    all_names = list(np.ravel(results))

    return jsonify(all_names)


@app.route("/api/v1.0/passengers")
def passengers():
    # Create our session (link) from Python to the DB
    session = Session(engine)

    """Return a list of passenger data including the name, age, and sex of each passenger"""
    # Query all passengers
    results = session.query(Passenger.name, Passenger.age, Passenger.sex).all()

    session.close()

    # Create a dictionary from the row data and append to a list of all_passengers
    all_passengers = []
    for name, age, sex in results:
        passenger_dict = {}
        passenger_dict["name"] = name
        passenger_dict["age"] = age
        passenger_dict["sex"] = sex
        all_passengers.append(passenger_dict)

    return jsonify(all_passengers)


if __name__ == '__main__':
    app.run(debug=True)


```


# Activity 11 - Student Turn - Chinook Database Analysis - 👩‍🎓👨‍🎓


# Chinook Database Analysis

In this activity, you will perform exploratory analysis of the Chinook database, which is a common database provided as a sample by many database tools.

## Instructions

* Create a Jupyter notebook for your analysis.

* Create a SQLAlchemy engine to the database `chinook.sqlite`.

* Use `automap_base` to reflect the database tables.

* Create references to the `invoices` and `invoice_items` tables, and call them `Invoices` and `Items`, respectively.

* Create a SQLAlchemy ORM session object.

* Design a query that lists all of the billing countries found in the invoices table.

* Design a query that lists the invoice totals for each billing country, and sort the output in descending order.

    * The results tuple should contain the country name and the invoice total for that country, using all records in the invoices table.

* Design a query that lists all of the billing postal codes for the USA.

* Calculate the item totals, using **sum(UnitPrice \* Quantity)**, for the USA.

    * Return the value as a scalar floating-point number.

* Calculate the invoice item totals using **sum(UnitPrice \* Quantity)** for each billing postal code in the USA.

---


In [3]:
# Ignore SQLITE warnings related to Decimal numbers in the Chinook database
import warnings
warnings.filterwarnings('ignore')
# Import Dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func

In [4]:
# Create an engine for the chinook.sqlite database
engine = create_engine("sqlite:///./resources/chinook.sqlite", echo=False)



In [5]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(autoload_with=engine)
Base.classes.keys()

['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoice_items',
 'invoices',
 'tracks',
 'media_types',
 'playlists']

In [6]:
# Save a reference to the invoices table as `Invoices`
Invoices = Base.classes.invoices



In [7]:
# Create a database session object
session = Session(engine)

In [8]:
# List all of the countries found in the invoices table
session.query(Invoices.BillingCountry).group_by(Invoices.BillingCountry).all()


[('Argentina',),
 ('Australia',),
 ('Austria',),
 ('Belgium',),
 ('Brazil',),
 ('Canada',),
 ('Chile',),
 ('Czech Republic',),
 ('Denmark',),
 ('Finland',),
 ('France',),
 ('Germany',),
 ('Hungary',),
 ('India',),
 ('Ireland',),
 ('Italy',),
 ('Netherlands',),
 ('Norway',),
 ('Poland',),
 ('Portugal',),
 ('Spain',),
 ('Sweden',),
 ('USA',),
 ('United Kingdom',)]

In [9]:
# Alternative Solution using `distinct`
# session.query(Invoices.BillingCountry).distinct().all()


In [10]:
# Design a query that lists the invoices totals for each billing country 
# and sort the output in descending order.
session.query(Invoices.BillingCountry, func.sum(Invoices.Total)).\
    group_by(Invoices.BillingCountry).\
    order_by(func.sum(Invoices.Total).desc()).all()




[('USA', Decimal('523.06')),
 ('Canada', Decimal('303.96')),
 ('France', Decimal('195.10')),
 ('Brazil', Decimal('190.10')),
 ('Germany', Decimal('156.48')),
 ('United Kingdom', Decimal('112.86')),
 ('Czech Republic', Decimal('90.24')),
 ('Portugal', Decimal('77.24')),
 ('India', Decimal('75.26')),
 ('Chile', Decimal('46.62')),
 ('Ireland', Decimal('45.62')),
 ('Hungary', Decimal('45.62')),
 ('Austria', Decimal('42.62')),
 ('Finland', Decimal('41.62')),
 ('Netherlands', Decimal('40.62')),
 ('Norway', Decimal('39.62')),
 ('Sweden', Decimal('38.62')),
 ('Poland', Decimal('37.62')),
 ('Italy', Decimal('37.62')),
 ('Denmark', Decimal('37.62')),
 ('Australia', Decimal('37.62')),
 ('Argentina', Decimal('37.62')),
 ('Spain', Decimal('37.62')),
 ('Belgium', Decimal('37.62'))]

In [11]:
# Save a reference to the invoice_items table as `Items`
Items = Base.classes.invoice_items



In [12]:
# List all of the Billing Postal Codes for the USA.
results = session.query(Invoices.BillingPostalCode).\
    filter(Invoices.BillingCountry == 'USA').group_by(Invoices.BillingPostalCode).all()
results


[('10012-2612',),
 ('2113',),
 ('32801',),
 ('53703',),
 ('60611',),
 ('76110',),
 ('84102',),
 ('85719',),
 ('89503',),
 ('94040-111',),
 ('94043-1351',),
 ('95014',),
 ('98052-8300',)]

In [13]:
# Calculate the Item Totals (sum(UnitPrice * Quantity)) for the USA
session.query(func.sum(Items.UnitPrice * Items.Quantity)).\
    filter(Invoices.InvoiceId == Items.InvoiceId).\
    filter(Invoices.BillingCountry == 'USA').scalar()


Decimal('523.0600000000')

In [14]:
# Calculate the Item Totals `sum(UnitPrice * Quantity)` for each Billing Postal Code in the USA
# Sort the results in descending order by Total
session.query(Invoices.BillingPostalCode, func.sum(Items.UnitPrice * Items.Quantity)).\
    filter(Invoices.InvoiceId == Items.InvoiceId).\
    filter(Invoices.BillingCountry == 'USA').\
    group_by(Invoices.BillingPostalCode).\
    order_by(func.sum(Items.UnitPrice * Items.Quantity).desc()).all()


[('76110', Decimal('47.6200000000')),
 ('60611', Decimal('43.6200000000')),
 ('84102', Decimal('43.6200000000')),
 ('53703', Decimal('42.6200000000')),
 ('94040-111', Decimal('39.6200000000')),
 ('98052-8300', Decimal('39.6200000000')),
 ('32801', Decimal('39.6200000000')),
 ('95014', Decimal('38.6200000000')),
 ('94043-1351', Decimal('37.6200000000')),
 ('89503', Decimal('37.6200000000')),
 ('85719', Decimal('37.6200000000')),
 ('2113', Decimal('37.6200000000')),
 ('10012-2612', Decimal('37.6200000000'))]